# <h1 align="center"><font color="gree">Building an Agentic System to enhance RAG with Self-Grading and Web Search Capabilities using Pydantic.AI, Groq and Langchain</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este notebook foi baseado no tutorial de [Plaban Nayak](https://medium.com/the-ai-forum/building-an-agentic-system-to-enhance-rag-with-self-grading-and-web-search-capabilities-using-3f9a1d885730).

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*I-kwOIjL3WrD3NL0WKLcYQ.jpeg)

In [12]:
import nest_asyncio
nest_asyncio.apply()

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
Eddy_key_openai  = os.environ['OPENAI_API_KEY']
Eddy_key_groq = os.environ['GROQ_API_KEY']
Eddy_key_gemini = os.environ['GEMINI_API_KEY']
Eddy_key_tavily = os.environ['TAVILY_API_KEY'] # https://tavily.com/

In [13]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.models.groq import GroqModel

openai_model = OpenAIModel('gpt-4o-mini')
groq_model = GroqModel("llama-3.3-70b-versatile")

In [14]:
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("./data/FIBROMIALGIA.pdf")
documents = loader.load()

split_docs = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(documents)

split_docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-01T14:52:50+00:00', 'moddate': '2024-09-01T14:52:50+00:00', 'source': './data/FIBROMIALGIA.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='Manejo da Fibromialgia: Uma Revisão de Literatura \nMateus Fonseca Dumont et. Al. \nBrazilian Journal of Implantology and Health Sciences \nVolume 6, Issue 9 (2024), Page 31-40. \n \n \n \n \n \n \n \nMANEJO DA FIBROMIALGIA: UMA  REVISÃO DE \nLITERATURA \nMateus Fonseca Dumont 1, Ramon Sousa Dias 2, Raianny Maria Pessoa \nVeiga2, Iasmyn Madalena de Carvalho Sandes Araujo2, Maxwellen Cabral De \nAndrade2, Kaio Mascarenhas Ferraz da Silva 2, Vitória Maria dos Passos'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-01T14:52:50+00:00', 'moddate': '2024-09-01T14:52:50+00:00', 'source': './data/FIBROM

In [15]:
len(split_docs)

41

In [16]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large" # "sentence-transformers/all-MiniLM-L6-v2"
                                 )

persist_directory = "./chroma_db"
#Build Index:
vectorstore = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory,
    collection_name="fibromyalgia"
)

In [17]:
from dataclasses import dataclass

@dataclass
class Deps:
    question:str | None
    context:str | None

In [24]:
import nest_asyncio
nest_asyncio.apply()

groq_agent = Agent(groq_model,
                   deps_type=Deps,
                    retries=2,
                    result_type=str,
                   system_prompt="""Você é um assistente útil e proficiente em responder de forma concisa, objetiva e objetiva a perguntas feitas com base no contexto fornecido.
                   Você tem que usar o `retrievre_tool` para obter o contexto relevante e gerar uma resposta baseada no contexto recuperado
                   Você é um assistente de avaliação. Avalie a resposta com base em:
        1. Relevância à pergunta
        2. Fidedignidade ao contexto
        3. Qualidade e completude do contexto
        
        Por favor, avalie a seguinte resposta com base em:
        1. Relevância (0-1): Quão bem ela responde à pergunta?
        2. Fidedignidade (0-1): Quão bem ela se mantém ao contexto fornecido?
        3. Qualidade do contexto (0-1): Quão completo e relevante é o contexto fornecido?
        
        Question: {ctx.deps.query}
        Context: {ctx.deps.context}
        Answer: {ctx.deps.response}
        
        Também determine se uma busca na web é necessária para aumentar o contexto.
        
        Forneça as notas e explicações no formato JSON com os atributos chave 'Relevancy','Faithfulness','Context Quality','Needs Web Search':
        {
            "Relevancy": <pontuação>,
            "Faithfulness": <pontuação>,
            "Context Quality": <pontuação>,
            "Needs Web Search": <true/false>,
            "Explanation": <explicação>,
            "Answer":<forneça a resposta com base no contexto do `retrievre_tool` se o valor de 'Need Web Search' for 'false' caso contrário, use a função `websearch_tool` para gerar a resposta final
        }
        """
        )

In [25]:
from tavily import TavilyClient


@groq_agent.tool_plain
async def websearch_tool(question) -> str:  
    """Verifique se o quadrado é um vencedor"""
    # Step 1. Instanciando o TavilyClient:
    tavily_client = TavilyClient(api_key=Eddy_key_tavily)

    # Step 2. Executando uma consulta de busca Q&A:
    answer = tavily_client.qna_search(query=question)

    # Step 3. Isso está resolvido! Sua pergunta foi respondida!
    print(f"BUSCA NA WEB:{answer}")
    return answer

In [26]:
from typing import List
from pydantic_ai import RunContext


@groq_agent.tool
async def rertiever_tool( ctx: RunContext[Deps],question:str)-> List[str]:
  load_vectorstore = Chroma(persist_directory=persist_directory,
                            embedding_function=embedding,
                            collection_name="fibromyalgia"
                           )
  docs = load_vectorstore.similarity_search(question,k=3)
  documnets = [d.page_content for d in docs]
  print(f"RAG Retrieval:{documnets}")
  return documnets

In [27]:
query = "O que é Fibromialgia?"

response = groq_agent.run_sync(query)

response

RAG Retrieval:['REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética \ngeneralizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve \ndisfunção no sistema nervoso central, com influências genéticas, ambientais e \npsicológicas. O manejo da fibromial gia exige uma abordagem multidisciplinar, que inclui \ntratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados', 'REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética \ngeneralizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve \ndisfunção no sistema nervoso central, com influências genéticas, ambientais e \npsicológicas. O manejo da fibromial gia exige uma abordagem multidisciplinar, que inclui \ntratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados', 'REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma 

AgentRunResult(data='{\n    "Relevancy": 0.8,\n    "Faithfulness": 0.9,\n    "Context Quality": 0.7,\n    "Needs Web Search": false,\n    "Explanation": "A resposta fornecida fornece uma visão geral sobre a fibromialgia, suas características principais e as abordagens de manejo. No entanto, a resposta poderia ser mais detalhada e específica para atender completamente às necessidades de informação do usuário.",\n    "Answer": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas. O manejo da fibromialgia exige uma abordagem multidisciplinar, que inclui tratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados com exercícios físicos regulares e terapias comportamentais."\n}')

In [28]:
print(response.data)

{
    "Relevancy": 0.8,
    "Faithfulness": 0.9,
    "Context Quality": 0.7,
    "Needs Web Search": false,
    "Explanation": "A resposta fornecida fornece uma visão geral sobre a fibromialgia, suas características principais e as abordagens de manejo. No entanto, a resposta poderia ser mais detalhada e específica para atender completamente às necessidades de informação do usuário.",
    "Answer": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas. O manejo da fibromialgia exige uma abordagem multidisciplinar, que inclui tratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados com exercícios físicos regulares e terapias comportamentais."
}


In [30]:
# Query 2:
query = "O que é Fibromialgia e quais são as causas?"
response = groq_agent.run_sync(query)

response

RAG Retrieval:['REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética \ngeneralizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve \ndisfunção no sistema nervoso central, com influências genéticas, ambientais e \npsicológicas. O manejo da fibromial gia exige uma abordagem multidisciplinar, que inclui \ntratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados', 'REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética \ngeneralizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve \ndisfunção no sistema nervoso central, com influências genéticas, ambientais e \npsicológicas. O manejo da fibromial gia exige uma abordagem multidisciplinar, que inclui \ntratamentos farmacológicos como antidepressivos e anticonvulsivantes, combinados', 'REVISÃO DE LITERATURA \n \nRESUMO \n \nA fibromialgia é uma 

AgentRunResult(data='{\n    "Relevancy": 0.8,\n    "Faithfulness": 0.9,\n    "Context Quality": 0.7,\n    "Needs Web Search": false,\n    "Explanation": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas.",\n    "Answer": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. As causas incluem disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas."\n}')

In [32]:
print(response.data)

{
    "Relevancy": 0.8,
    "Faithfulness": 0.9,
    "Context Quality": 0.7,
    "Needs Web Search": false,
    "Explanation": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas.",
    "Answer": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. As causas incluem disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas."
}


In [35]:
from langchain_core.output_parsers import JsonOutputParser

print(response.data)
parser = JsonOutputParser()
print(parser.parse(response.data))
print("\n")
print(parser.parse(response.data)['Answer'])
#print(response.cost())

{
    "Relevancy": 0.8,
    "Faithfulness": 0.9,
    "Context Quality": 0.7,
    "Needs Web Search": false,
    "Explanation": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas.",
    "Answer": "A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. As causas incluem disfunção no sistema nervoso central, com influências genéticas, ambientais e psicológicas."
}
{'Relevancy': 0.8, 'Faithfulness': 0.9, 'Context Quality': 0.7, 'Needs Web Search': False, 'Explanation': 'A fibromialgia é uma síndrome crônica caracterizada por dor musculoesquelética generalizada, fadiga persistente e sensibilidade aumentada. A fisiopatologia envolve disfunção no sistema nervoso central, com influências genéticas, ambientais 